## patsy 패키지
- 회귀분석 전처리를 위한 패키지
- 인코딩, 변환 등을 쉽게 해주는 기능을 제공

In [1]:
from patsy import dmatrix

In [2]:
np.random.seed(0)
x1 = np.random.rand(5) + 10
x2 = np.random.rand(5) * 10
y = x1 + 2 * x2 + np.random.randn(5)
df = pd.DataFrame(np.array([x1, x2, y]).T, columns=["x1", "x2", "y"])

In [3]:
df.tail()

,x1,x2,y
0,10.548814,6.458941,23.610739
1,10.715189,4.375872,20.921207
2,10.602763,8.917730,29.199261
3,10.544883,9.636628,29.939813
4,10.423655,3.834415,18.536348


##### 자동 augmentation
- intercept column으로 자동 추가

In [4]:
dmatrix("x1", df)

DesignMatrix with shape (5, 2)
  Intercept        x1
          1  10.54881
          1  10.71519
          1  10.60276
          1  10.54488
          1  10.42365
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)

In [6]:
# 상수항 제외1
dmatrix("x1 - 1", df)

DesignMatrix with shape (5, 1)
        x1
  10.54881
  10.71519
  10.60276
  10.54488
  10.42365
  Terms:
    'x1' (column 0)

In [7]:
# 상수항 제외2
dmatrix("x1 + 0", df)

DesignMatrix with shape (5, 1)
        x1
  10.54881
  10.71519
  10.60276
  10.54488
  10.42365
  Terms:
    'x1' (column 0)

In [8]:
# 두개 이상 데이터는 +로 연결
dmatrix("x1 + x2", df)

DesignMatrix with shape (5, 3)
  Intercept        x1       x2
          1  10.54881  6.45894
          1  10.71519  4.37587
          1  10.60276  8.91773
          1  10.54488  9.63663
          1  10.42365  3.83442
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'x2' (column 2)

In [9]:
dmatrix("x1 + x2 - 1", df)

DesignMatrix with shape (5, 2)
        x1       x2
  10.54881  6.45894
  10.71519  4.37587
  10.60276  8.91773
  10.54488  9.63663
  10.42365  3.83442
  Terms:
    'x1' (column 0)
    'x2' (column 1)

In [10]:
# 두 변수를 곱하여 새로운 변수로 추가
dmatrix("x1 + x2 + x1:x2 - 1", df)

DesignMatrix with shape (5, 3)
        x1       x2      x1:x2
  10.54881  6.45894   68.13417
  10.71519  4.37587   46.88830
  10.60276  8.91773   94.55258
  10.54488  9.63663  101.61711
  10.42365  3.83442   39.96862
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

In [11]:
dmatrix("x1 * x2 - 1", df)

DesignMatrix with shape (5, 3)
        x1       x2      x1:x2
  10.54881  6.45894   68.13417
  10.71519  4.37587   46.88830
  10.60276  8.91773   94.55258
  10.54488  9.63663  101.61711
  10.42365  3.83442   39.96862
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

##### 변환
- center(x) : 평균제거
- standardize(x) : 평균제거 및 표준편차로 스케일링 (정규화)
- scale(x) : 표준편차로 스케일링

In [12]:
dmatrix("x1 + np.log(np.abs(x2))", df)

DesignMatrix with shape (5, 3)
  Intercept        x1  np.log(np.abs(x2))
          1  10.54881             1.86547
          1  10.71519             1.47611
          1  10.60276             2.18804
          1  10.54488             2.26557
          1  10.42365             1.34402
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'np.log(np.abs(x2))' (column 2)

In [13]:
def doubleit(x):
    return 2 * x

dmatrix("doubleit(x1)", df)

DesignMatrix with shape (5, 2)
  Intercept  doubleit(x1)
          1      21.09763
          1      21.43038
          1      21.20553
          1      21.08977
          1      20.84731
  Terms:
    'Intercept' (column 0)
    'doubleit(x1)' (column 1)

In [14]:
dmatrix("center(x1) + standardize(x1) + scale(x2)", df)

DesignMatrix with shape (5, 4)
  Intercept  center(x1)  standardize(x1)  scale(x2)
          1    -0.01825         -0.19319   -0.07965
          1     0.14813          1.56828   -0.97279
          1     0.03570          0.37799    0.97458
          1    -0.02218         -0.23480    1.28282
          1    -0.14341         -1.51828   -1.20495
  Terms:
    'Intercept' (column 0)
    'center(x1)' (column 1)
    'standardize(x1)' (column 2)
    'scale(x2)' (column 3)

##### 진짜  더한 값을 column으로 추가할 때 : I()

In [15]:
dmatrix("I(x1 + x2)", df)

DesignMatrix with shape (5, 2)
  Intercept  I(x1 + x2)
          1    17.00775
          1    15.09106
          1    19.52049
          1    20.18151
          1    14.25807
  Terms:
    'Intercept' (column 0)
    'I(x1 + x2)' (column 1)

In [16]:
from sklearn.datasets import load_boston

boston = load_boston()
dfX0_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
# dfX_boston = sm.add_constant(dfX0_boston)
dfy_boston = pd.DataFrame(boston.target, columns=["MEDV"])
df_boston = pd.concat([dfX0_boston, dfy_boston], axis=1)

In [21]:
model = sm.OLS.from_formula("MEDV ~ CRIM + ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + C(CHAS) - 1", df_boston)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sat, 30 Jun 2018   Prob (F-statistic):          6.95e-135
Time:                        14:59:25   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
C(CHAS)[0.0]    36.4911      5.104      7.149   

In [17]:
from sklearn.datasets import load_boston

boston = load_boston()
dfX0_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
dfX_boston = sm.add_constant(dfX0_boston)
dfy_boston = pd.DataFrame(boston.target, columns=["MEDV"])
df_boston = pd.concat([dfX_boston, dfy_boston], axis=1)

In [19]:
model = sm.OLS.from_formula("MEDV ~ CRIM + ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", df_boston)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Sun, 01 Jul 2018   Prob (F-statistic):          6.95e-135
Time:                        17:17:23   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4911      5.104      7.149      0.0

- 찰스강 유역이 아닌 집 가격모형은 상수항이  36.4911
- 찰스강 유역의 집 가격모옇은 36.4911 + 2.6886